К вашей компании пришел заказчик, которому нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, чтобы вы оценили возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных вам не предоставляется. Требуется, чтобы качество работы вашего алгоритма (по accuracy) было строго больше 85%.

Оценка качества в этом задании реализована через контест на Kaggle Inclass:

https://www.kaggle.com/c/morecomplicatedsentiment

Вам предстоит посмотреть на предоставленные заказчиком отзывы, собрать похожие отзывы в качестве обучающей выборки, и поэкспериментировать с постановкой задачи (разметкой вашей выборки на позитивные и негативные примеры) так, чтобы результат на примерах заказчика был по возможности получше.

Обратите внимание, что заказчик предоставил всего 100 примеров в качестве тестовой выборки - ситуация, когда размеченных данных почти нет - вообще очень частая в индустриальном анализе данных. Конечно, эти отзывы можно было бы идеально разметить вручную и получить максимальное качество, но вы сами не заинтересованы в таком подходе, т.к. потом придется и на всех новых примерах демонстрировать заказчику идеальную работу, что, конечно, вряд ли будет по силам алгоритму. В любом случае рано или поздно алгоритм придется разрабатывать, поэтому попытки "сжульничать" и не делать никакой модели не одобряются.

В качестве ответа в этом задании вам нужно загрузить ноутбук с решением и скриншот вашего результата на leaderboard.

Убедитесь, что:

1) ход вашего решения задокументирован достаточно подробно для того, чтобы ваши сокурсники поняли, что вы делали и почему;

2) ваша команда в соревновании состоит только из вас и названа вашим логином на Stepik, чтобы ваши сокурсники могли понять, что на скриншоте именно ваш результат.

In [486]:
import pandas as pd
from bs4 import BeautifulSoup,NavigableString,Tag
import requests


In [487]:
#прочитаем данные по тестовой выборке

r = open("test.txt",encoding="utf8")
soup = BeautifulSoup(r.read())
print(soup)

<html><body><review>Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.
Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.
</review>
<review>ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если 
постоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается.
 Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -после нее набор с

судя по тексту это отзывы на сотовые телефоны. соберем их в таблицу текстов.


In [488]:
test_review_list = []

In [489]:
for node in soup.findAll('review'):
    s = str(node.findAll(text=True))
    test_review_list.append(s.replace("\\n", " "))


Ччтобы обучить модель, будем собирать статистику с сайтов с аналогичным отзывами. Например с сайта https://www.e-katalog.ru


In [498]:
def get_rev(url):
    r  = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data)

    divTag = soup.find_all("div", {"class": "disadv__text disadv__plus"})
    for tag in divTag:
        positive_r.append(tag.text.replace("\\n", " "))
    
    divTag = soup.find_all("div", {"class": "disadv__text disadv"})
    for tag in divTag:
        negative_r.append(tag.text.replace("\\n", " "))

    

In [499]:
positive_r = []
divTag = soup.find_all("div", {"class": "disadv__text disadv__plus"})
for tag in divTag:
       positive_r.append(tag.text.replace("\\n", " "))
    


In [515]:
positive_r = []
negative_r = []

get_rev("https://www.e-katalog.ru/brand-r/122/SAMSUNG/mobilnye-telefony-samsung-otzyvy/")
get_rev("https://www.e-katalog.ru/brand-r/122/XIAOMI/mobilnye-telefony-xiaomi-otzyvy/")
get_rev("https://www.e-katalog.ru/brand-r/122/APPLE/mobilnye-telefony-apple-otzyvy/")
get_rev("https://www.e-katalog.ru/brand-r/122/HUAWEI/mobilnye-telefony-huawei-otzyvy/")
get_rev("https://www.e-katalog.ru/brand-r/122/ONEPLUS/mobilnye-telefony-oneplus-otzyvy/")
get_rev("https://www.e-katalog.ru/brand-r/122/NOKIA/mobilnye-telefony-nokia-otzyvy/")





In [516]:
print(len(negative_r))
print(len(positive_r))


250
265


Добавим отзывов с яндекс-маркета

In [517]:
def get_reviews_from_yandex_market(url):
    r  = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data)
    print(soup)
    divTag = soup.find_all("dl")
    print(len(divTag))
    for tag in divTag:
        #positive_r.append(tag.text.replace("\\n", " "))
        for rtag in divTag:
            ddTag = rtag.find('dd',text=True)
            dtTag = rtag.find('dt',text=True)
            if dtTag!= None and dtTag.text == "Достоинства: ":
                s = ddTag.text.replace('\\n',' ')
                positive_r.append(s.replace('\r\n'," "))
            elif  dtTag!= None and dtTag.text == "Недостатки: ":
                s = ddTag.text.replace('\\n',' ')
                negative_r.append(s.replace('\r\n'," "))
            
            

In [518]:
url = "https://market.yandex.ru/catalog--mobilnye-telefony-otzyvy-pokupatelei/69249/list?show-reviews=1&glfilter=4940921%3A13475069&local-offers-first=0&onstock=0&page=19"

In [519]:
get_reviews_from_yandex_market(url)

<!DOCTYPE html>
<!--BEGIN [@MarketNode/Root] / static--><html class="i-ua_js_no i-ua_css_standard i-ua_os-family_unknown i-ua_browser-engine_unknown i-ua_browser-name_unknown i-ua_placeholder_yes" lang="ru" prefix="og: http://ogp.me/ns#"><head><script nonce="qk2RtwIBnHqiz8Ab/flFJA==">document.cookie = 'js=1;path=/';</script><style>
        .i-a11y-reset-button {
            width: auto;
            margin: 0;
            padding: 0;
            border: none;

            font: inherit;
            line-height: inherit;
            text-rendering: inherit;
            letter-spacing: inherit;
            word-spacing: inherit;
            text-transform: inherit;
            text-indent: inherit;
            text-shadow: inherit;
            text-align: inherit;
            color: inherit;

            overflow: visible;

            cursor: pointer;

            background: transparent;

            -webkit-font-smoothing: inherit;
            -moz-osx-font-smoothing: inherit;

       

In [520]:
print(len(negative_r))
print(len(positive_r))


670
685


Теперь обучаем модель

In [522]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt


In [523]:
l_p = (['pos'] * len(positive_r))
l_n = (['neg'] *len(negative_r))
labels = l_p + l_n
texts = positive_r + negative_r

In [524]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [525]:
import warnings
warnings.filterwarnings('ignore')

for clf in [LogisticRegression, LinearSVC, SGDClassifier,MultinomialNB,RandomForestClassifier]:
    print(clf)
    print(cross_val_score(text_classifier(CountVectorizer(), clf()), texts, labels).mean())
    print ("\n")

<class 'sklearn.linear_model.logistic.LogisticRegression'>
0.9079781827318575


<class 'sklearn.svm.classes.LinearSVC'>
0.9057706772130937


<class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
0.8549980502815263


<class 'sklearn.naive_bayes.MultinomialNB'>
0.8726450419230262


<class 'sklearn.ensemble.forest.RandomForestClassifier'>
0.871150529687115




In [526]:
#логистическая регрессия, смотрим с каким векторайзером лучше
warnings.filterwarnings('ignore')
clf_b = LogisticRegression(C=70, dual=True)
vectorizer = CountVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(texts)
clf_b.fit(X,labels)
print(cross_val_score(clf_b,X, labels, cv=10).mean()) 


clf_b = LogisticRegression(C=70, dual=True)
vectorizer =  TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(texts)
clf_b.fit(X,labels)
print(cross_val_score(clf_b,X, labels, cv=10).mean()) 


0.9272058823529411
0.9389705882352942


In [527]:
#окончательный выбор за TfidfVectorizer(ngram_range=(1,2))
clf_b .fit(X,labels)
X_test = vectorizer.transform(test_review_list)
predict_vector = clf_b.predict(X_test)
data_predict = pd.DataFrame({'Id': list(range(0, len(predict_vector))),'y': predict_vector})
data_predict.head()
export_csv = data_predict.to_csv (r'submit_f2.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
